# Feature Store 

In this Notebook we will take our dataset and place it into the feature store.

Note: In order to execute this Notebook your Sagemaker Execution Role must be granted the **AmazonS3FullAccess** policy, or a more restricted version for access to the S3 bucket you will use for your feature store.

In [ ]:
import utils.display as disp
import utils.config as cfg
import pandas as pd


In [ ]:
path_to_partitioned = cfg.get_path_to_partitioned_data()
train_data = path_to_partitioned + "/train.csv"
train_df = pd.read_csv(train_data)
train_df.head()

In [ ]:
import boto3
import sagemaker
from sagemaker.session import Session


region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

# S3 Bucket for Offline Storage

The Feature Store requires an S3 bucket for offline store of your features.

They will be structured with meta-data, and made available to query using Athena SQL. 

In [ ]:
# You can modify the following to use a bucket of your choosing
s3_client = boto3.client('s3', region_name=region)
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = 'sagemaker-featurestore-demo'

print(default_s3_bucket_name)

In [ ]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print (role)


In [ ]:
from time import gmtime, strftime, sleep

feature_group_name = 'patient-feature-group-' + strftime('%d-%H-%M-%S', gmtime())

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=feature_store_session)

In [ ]:

def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == 'object':
            data_frame[label] = data_frame[label].astype("str").astype("string")

# cast object dtype to string. The SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
cast_object_to_string(train_df)


In [ ]:
import time
current_time_sec = int(round(time.time()))

# record identifier and event time feature names
record_identifier_feature_name = "encounter_id"
event_time_feature_name = "RecordTime"

# append EventTime feature
train_df[event_time_feature_name] = pd.Series([current_time_sec]*len(train_df), dtype="float64")


In [ ]:
train_df.head()

In [ ]:
# load feature definitions to the feature group. 
# SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.

feature_group.load_feature_definitions(data_frame=train_df); # output is suppressed


In [ ]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")


feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True
)

wait_for_feature_group_creation_complete(feature_group=feature_group)

# Inspect the feature group

In [ ]:
feature_group.describe()

In [ ]:
 # use boto client to list FeatureGroups
sagemaker_client.list_feature_groups()


In [ ]:
feature_group.ingest(
    data_frame=train_df, max_workers=3, wait=True
)

In [ ]:
record_identifier_value = str(112487478)

record = featurestore_runtime.get_record(FeatureGroupName=feature_group_name, RecordIdentifierValueAsString=record_identifier_value)

In [ ]:
print(record)

In [ ]:
print(feature_group.as_hive_ddl())

In [ ]:
account_id = boto3.client('sts').get_caller_identity()["Account"]
print(account_id)

feature_group_s3_prefix = prefix + '/' + account_id + '/sagemaker/' + region + '/offline-store/' + feature_group_name + '/data'

offline_store_contents = None
while (offline_store_contents is None):
    objects_in_bucket = s3_client.list_objects(Bucket=default_s3_bucket_name,Prefix=feature_group_s3_prefix)
    if ('Contents' in objects_in_bucket and len(objects_in_bucket['Contents']) > 1):
        offline_store_contents = objects_in_bucket['Contents']
    else:
        print('Waiting for data in offline store...\n')
        sleep(60)

print('Data available.')



# Retrieve the data

Feature Store makes the data available in Glue Data Catalog that can be queried using Athena.

In this example we are just selecting a subset of the columns. 
However we could just as easily join multiple feature groups together to form a more comprehensive data set.

In [ ]:
_query = feature_group.athena_query()

_table = _query.table_name

query_string = 'SELECT readmitted, admission_source_id, number_diagnoses, num_medications, number_inpatient, number_outpatient  FROM "'+_table+'" '
print('Running ' + query_string)

# run Athena query. The output is loaded to a Pandas dataframe.
#dataset = pd.DataFrame()
_query.run(query_string=query_string, output_location='s3://'+default_s3_bucket_name+'/'+prefix+'/query_results/')
_query.wait()
dataset = _query.as_dataframe()

dataset

### Get S3 path for ML jobs

Sagemaker ML Jobs expect to retrive data from S3.

Luckily, the query results were put directly into S3 for us.
We just need to build the path to our query result.

In [ ]:
# Prepare query results for training.
query_execution = _query.get_query_execution()
query_result = 's3://'+default_s3_bucket_name+'/'+prefix+'/query_results/'+query_execution['QueryExecution']['QueryExecutionId']+'.csv'
print(query_result)
